# Reporte de problemas fitosanitarios en plantaciones de agave
--------------------

## Equipo 36

| Nombre | Matrícula |
| ------ | --------- |
| André Martins Cordebello | A00572928 |
| Enrique Eduardo Solís Da Costa | A00572678 |
| Delbert Francisco Custodio Vargas | A01795613 |

## Avance 3: baseline

### Instrucciones
Este avance consiste en construir un modelo de referencia que permita evaluar la viabilidad de la solución plantada. Si el baseline tiene un rendimiento similar al azar, podría indicar que el problema es intrínsecamente difícil o que los datos no contienen suficiente información para predecir el objetivo. De lo contrario, el baseline podría funcionar como una solución mínima aceptable cuando se trabaja en escenarios donde incluso un modelo simple puede proporcionar valor.

Un baseline facilita también la gestión de expectativas, tanto dentro del equipo como con los stakeholders, pues proporciona una comprensión inicial de lo que se puede lograr con métodos simples antes de invertir tiempo y recursos en enfoques más complejos.

# ChatBot para reporte de infestaciones fitosanitarias en plantaciones de Agave

Para conectar nuestro LLM entrenado sobre una porción de datos de nuestros `text_features`, es necesario crear una conexión hacia WhatsApp.

Respecto a la conexión hacia WhatsApp, es importante mencionar que Meta maneja criterios bastante estrictos para poder hacer un uso directo de su API. Estos criterios se pueden resumir en lo siguiente:

- Se debe contar con una cuenta de Meta Business y ser un integrador verificado. Este proceso puede demorar semanas y por lo tanto, no es buena idea llevarlo a cabo para el prototipo.
- Luego de estar verificado, se pueden hacer uso de distintas formas de mensajes:
  - Freeform Messages: son aquellos que permiten enviar y recibir texto libre de parte de los usuarios.
  - Template Messages: son los mensajes que permiten enviar opciones predefinidas hacia un usuario. Estos deben ser aprobados por Meta.
  - Auth Messages: son notificaciones para enviar códigos para iniciar sesión de 2 Factores, o parecido.

Con lo anterior, debido a que no contamos con el tiempo suficiente para llevar a cabo un registro, pago y posterior entrevista de verificación de Meta, en esta entrega haremos uso de algunos modelos LLM locales para comparar las respuestas y seleccionar el modelo que nos servirá de base (Baseline) para llevar a cabo el fine-tuning del mismo.

Con esto, haremos pruebas con los siguientes modelos en este Notebook:

- `Mistral-7B`
- `Llama-3-8B`
- `Microsoft-Phi-3`

Por último, como cada modelo abarca casi por completo la memoria RAM de nuestra GPU, es necesario que durante el run de nuestra Baseline se libere esta memoria para asegurar que la Notebook pueda funcionar correctamente.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.language_models.llms import LLM
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import Field
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from typing import Optional, List, Any
from langchain.schema import Document
from transformers import pipeline
from peft import PeftModel
import gradio as gr
import pandas as pd
import gradio as gr
import numpy as np
import json
import torch
import gc
import re
import os

d:\Python_Envs\torch311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Python_Envs\torch311\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Comparación de LLMs sin fine-tuning

Para esta sección es importante mencionar la obteción de estos modelos:

- Primero, es necesario contar con una cuenta de HuggingFace.
- Segundo, se debe contar con el `auth_token` generado de HuggingFace para acceder a la descarga de estos modelos.
- Tercero, se puede hacer uso del script `download_llm.py` para descargar los modelos que se quieran recrear en este Notebook.
- Cuarto, es posible revisar el archivo `requirements.txt` para instalar las librerías necesarias con las versiones compatibles para que este código funcione correctamente.
- Quinto, por medio de Gradio es posible simular o ver una interfaz `Query-Result`, de forma que podamos evaluar la forma de contestar de cada modelo. Esto lo usaremos con el modelo que mejor responda en español (esta selección será subjetiva).

Con esto,  los prompts que usaremos son los siguientes:

- `Describe al gorgojo del agave`
- `Describe la planta de agave azul`
- `Describe como es un predio de agave azul y que riesgos presenta`

Bajo las siguientes condiciones:

- Que responda en una cantidad máxima de 128 tokens
- La cantidad mínima de tokens a generar será de 64

La idea es encontrar algún modelo que presente la mejor respuesta en español, pero también debemos tomar en cuenta el apartado de memoria y rendimiento. Al trabajar con LLMs, la VRAM de nuestra GPU debe ser lo suficientemente grande para soportar correrlo, ya que de lo contrario corremos el riesgo de procesar las respuestas a nuestros prompt desde el CPU. Esto último es *demasiado lento* como para considerar trabajar desde este componente.

In [2]:
prompt_1 = "Describe al gorgojo del agave brevemente"
prompt_2 = "Describe la planta de agave azul brevemente"
prompt_3 = "Describe como es un predio de agave azul y cuales riesgos presenta brevemente"

### `Mistral-7B-Instruct-v0.3`

Algo importante sobre `Mistral-7B-Instruct-v0.3` es que se requiere de al menos 14GB de memoria dedicada en la GPU. Esto, para nosotros en este momento, no es lograble a menos de que se use un servicio como Google Colab Pro.

Por lo tanto, hemos decidido aplicar `quantization` a este modelo, que en resumen es "hacer más pequeño" este LLM. El trade-off es que perderemos capacidad de mantener el contexto, pero tendremos respuestas más rápidas.

Para dar un ejemplo, al usar el modelo `Mistral-7B` sin quantization, debíamos esperar entre 20 a 30 minutos por respuesta de cada prompt. Al usar `quantization` el tiempo de espera se redujo a segundos (incluso menos de 1 minuto por respuesta).

In [3]:
# Ruta local del modelo
model_path = "D:/LLM Models/Mistral-7B-Instruct-v0.3"

# Aplicamos quantization para hacer el modelo más pequeño
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Carga del modelo y tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [01:18<00:00, 26.30s/it]


Procedemos a crear la `pipeline()` y a verificar que esté corriendo el modelo en GPU.

In [4]:
# Usamos pipeline() para integrar el tokenizer, modelo y el task o tarea
# que debe llevar a cabo nuestro bot.

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    min_new_tokens=64,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

Device set to use cuda:0


#### Respuestas de `Mistral-7B-Instruct-V0.3`

In [5]:
response = pipe(prompt_1)
print(response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Describe al gorgojo del agave brevemente.

El algorojo del agave, también conocido como cochineal rojo o insecto carminativo, es una especie de insecto escarabajo que se encuentra en México y América Central. Es un pequeño escarabajo que vive en las plantas del agave. Los adultos son de color rojo brillante y tienen un cuerpo cubierto de pelos. Se utilizan sus escalas para obtener el colorante natural rojo denominado carminio, que se utiliza en la industria alimentaria, cosmética y


In [6]:
response = pipe(prompt_2)
print(response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Describe la planta de agave azul brevemente.
La planta de agave azul, científicamente conocida como Agave tequilana Weber azul, es una especie de agave nativa de México que se cultiva principalmente en los estados de Jalisco, Nayarit y Tamaulipas. Es una agave robusta con hojas azules-verdosas de hasta 1,2 metros de largo y 7 centímetros de ancho, con bordes fuertemente espinosos. La planta produce una flor de color amarillo


In [7]:
response = pipe(prompt_3)
print(response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Describe como es un predio de agave azul y cuales riesgos presenta brevemente.

Un predio de agave azul, comúnmente conocido como mezcalero, se encuentra en regiones secas y desérticas de México, especialmente en Oaxaca, Guerrero, y Durango. Los campos de agaves azules son caracterizados por sus plantaciones bien separadas con espacios abiertos para permitir el crecimiento de la planta. Las agaves azules pueden alcanzar una edad de hasta 28 años antes de ser recolectadas para fabricar el mezcal.

Sin


#### Liberacion de memoria RAM de nuestra GPU

Como solo contamos con 8GB de RAM en la GPU actual, es necesario liberar la misma. Como estamos usando torch + CUDA, es necesario mover el modelo al CPU y eliminarlo de memoria.

In [8]:
# Liberamos la memoria de nuestra GPU para evitar un overflow por mantener más de 
# un LLM en memoria

model.cpu() # Movemos el modelo al CPU
del model   # Eliminamos el modelo de memmoria

# Limpiamos la cache de Torch para evitar problemas
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

### `meta-llamaMeta-Llama-3-8B`

Usaremos el LLM desarrollado por Meta también. Este se caracteriza por contar con 8 mil millones de parámetros. Por lo tanto, debemos cuantizarlo también para evitar un overflow en la memoria de nuestra GPU.

In [9]:
# Ruta local del modelo
model_path = "D:/LLM Models/meta-llamaMeta-Llama-3-8B"

# Aplicamos quantization para hacer el modelo más pequeño
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Carga del modelo y tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 4/4 [01:25<00:00, 21.42s/it]


In [10]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    min_new_tokens=64,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

Device set to use cuda:0


#### Respuestas de `meta-llamaMeta-Llama-3-8B`

In [11]:
response = pipe(prompt_1)
print(response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Describe al gorgojo del agave brevemente en espanol
The Agave is a type of succulent plant, that is, it stores water in its leaves. The species grows in tropical and subtropical regions of the world.
These plants are found mainly in Mexico, where they are used to produce tequila and mezcal.
In addition to these alcoholic beverages, the agave fibers are used as ropes or for making clothing, among other things. It also has medicinal properties and can be eaten raw or cooked.
We know that there are several species of agave plants. Among them we find the Agave americana (commonly known as "century plant"), which can


In [12]:
response = pipe(prompt_2)
print(response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Describe la planta de agave azul brevemente. The Blue Agave Plant, or Agave Tequilana, is a succulent plant that has blueish green leaves with sharp spines on the edges of its leaves. It is native to Mexico and is grown in many other countries as well.
The plant can grow up to 5 feet tall and has a long lifespan of about 10 years. The agave plant produces flowers only once during its lifetime which then produce seeds that fall off the plant after flowering. These seeds germinate into new plants called pups which are smaller versions of their parent plants but still capable of growing into adult plants themselves if they receive enough water and nutrients


In [13]:
response = pipe(prompt_3)
print(response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Describe como es un predio de agave azul y cuales riesgos presenta brevemente.
The project is located in the north of Mexico, in a town called Tepehuanes. It is an area with an altitude of 1000 meters above sea level and it is surrounded by mountains that are very high. The region is characterized by being arid because there is little rain and most of the precipitation occurs during summer. This has caused that the vegetation of this region consists mainly of shrubs like sagebrush and cactus. There is also a large amount of trees such as pine, oak and cedar. In addition to these plants, there is a plant called Agave Azul (Agave tequilana)


In [ ]:
# Nuevamente eliminamos el modelo de la memoria y de la cache para hacer espacio al siguiente LLM

model.cpu()
del model

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

### `microsoft-phi-3-mini-4k-instruct`

Este modelo, que se considera como lightweight, fue posible cuantizarlo a una resolución mayor (8Bits). Por lo tanto, haremos uso del cuantizado en 8bits para llevar a cabo la carga del modelo ya que de esta forma se reducen menos las cualidades del mismo.

In [15]:
# Ruta local del modelo
model_path = "D:/LLM Models/microsoft-phi-3-mini-4k-instruct"

# Aplicamos quantization para hacer el modelo más pequeño
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

# Carga del modelo y tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config, # Descomentar si se quiere cuantizar
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:41<00:00, 20.90s/it]


In [16]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    min_new_tokens=64,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

Device set to use cuda:0


#### Respuestas de `microsoft-phi-3-mini-4k-instruct`

In [17]:
response = pipe(prompt_1)
print(response[0]["generated_text"])

Describe al gorgojo del agave brevemente. Gusano de la hoja (Dactylopius coccus). El gusano es un insecto escamoso que vive en las hojas y tallos de los agaves, especialmente el Agave americana; mide aproximadamente 4 mm x 2 mm. La larva se alimenta de células parenquimatosas produciendo una mancha amarilla o blanca característica a lo largo del tallo interno cuando crece demasiado grande para adaptarse dentro del tejido vegetal. Las plantas infectadas por este á


In [18]:
response = pipe(prompt_2)
print(response[0]["generated_text"])

Describe la planta de agave azul brevemente en 50 palabras.
Agave americana 'Blue Glow' es una variedad ornamental con hojas grandes y coloridas que se utilizan a menudo como plantas decorativas debido a su belleza estética única, especialmente durante el otoño cuando las hojas tiñen un tono brillante azulado.


In [19]:
response = pipe(prompt_3)
print(response[0]["generated_text"])

Describe como es un predio de agave azul y cuales riesgos presenta brevemente. La planta de agave Azul, conocida por su nombre científico Agave tequilana var. azul o A. salmiana, se cultiva principalmente en el estado mexicano de Jalisco para la producción del Tequila. Este predio debe cumplir con ciertas condiciones ambientales específicas que favorecen el crecimiento óptimo de esta especie:

- Clima semiárido subtropical caracterizado por veranos calurosos e inviernos templados.
- Suelo arenoso bien drenado a base


In [ ]:
model.cpu()
del model

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

### Selección de LLM

| Modelo | Ventajas | Desventajas |
| ------ | -------- | ----------- |
| `Mistral-7B-Instruct-v0.3`  | Da respuestras bastante estructuradas y concisas. El modelo responde bien en Español. | Es un LLM pesado, ocupa aproximadamente 30GB de espacio en disco y debemos cuantizarlo en 4bits. Esto generó una reducción del 75% del modelo completo. |
| `meta-llamaMeta-Llama-3-8B` | El modelo es más rápido que `Mistral-7B`. | Se debe cuantizar, y por lo tanto puede perder granularidad en sus respuestas o textos generados. Otra desventaja es que los prompts en Español los respondió en Inglés, lo que indica que el proceso de fine-tuning tendrá que involucrar demasiados textos en español| 
| `microsoft-phi-3-mini-4k-instruct` | Es más ligero que `Mistral-7B` y `LLama-3-8B`. Responde en español consisamente. | No es maneja un contexto tan especializado como `Mistral-7B` o `Llama-3-8B`. |


Por lo tanto, tomando en cuenta los resultados anteriores, decidimos trabajar sobre el modelo `Microsoft-phi-3-mini-4k-instruct` por lo siguiente:

- Es un modelo ligero que responde bien en español. 
- Sus respuestas cargaron más rápido que las de `Mistral-7B`, lo que es una ventaja al tener  un ChatBot.
- Existe documentación amplia sobre como llevar a cabo el fine-tuning de este modelo.
- Al tener una arquitectura `instruct`, es posible entrenarlo con archivos `jsonl` los cuales contengan diccionarios con valores de `instruction`, `input` y `output`.
- Aunque `Mistral-7B` fue el modelo que mejor rendimiento tuvo, `Phi-3` ocupa solamente el 23% del espacio en disco de lo que `Mistral-7B` ocupa.  Esto hace que `Phi-3` sea un modelo  más portable al poder correr en hardware más débil (como es en nuestro caso). Esto reduce el costo de montar un ChatBot nativo por medio de un LLM, y permite justificar el no desarrollar un wraper de APIs de marcas como OpenAI y Claude.

## Generación de archivos para fine-tuning

Ahora que elegimos nuestro modelo (`Phi-3`), debemos generar los archivos necesarios para llevar a cabo el fine-tuning del mismo.

Para esto, Daniel Voigt Godoy publicó su libro `A Hands-On Guide to Fine-Tuning Large Language Models with PyTorch and Hugging Face` (Voigt, 2025), en el cual generó un ejemplo de cómo llevar a cabo el fine-tuning de un LLM.

Por lo tanto, hicimos uso del código de Daniel Voigt en el notebook `LL-Fine-Tuning.ipynb`, en el cual:

- Cargamos el modelo `Phi-3` de Microsoft
- Por medio de librerías de HuggingFace se lleva a cabo un proceso conocido como LoRA (Low Rank Adaptation) para *cambiar solo los pesos de las capas finales del modelo a hacer fine-tuning`.

Lo anterior permitió "especializar" a Phi-3 con base en nuestro dataset trabajado en entregas anteriores. Por lo anterior, todo el proceso de Fine-Tuning se encuentra en la notebook mencionada anteriormente, y en este notebook solo haremos uso del modelo obtenido por este proceso.

Por último, quisiéramos aclarar que recomendamos altamente correr el notebook `LL-Fine-Tuning.ipynb` en Google Colab u otro servicio que ofrezca GPUs potentes. Como ejemplo, al querer  llevar a cabo el fine-tuning localmente, debíamos esperar aproximadamente 3 días; pero al usar Google Colab en horarios donde pocos usuarios se conectan, fue posible llevar a cabo el fine-tuning en 5 horas. Algo  importante es que solo llevamos a cabo el proceso de fine-tuning usando información de Julio de 2025 en nuestro dataset.

Ahora bien, procedemos con la generación de archivos `jsonl` que servirán para hacer el fine-tuning the `Phi-3`.

In [20]:
# Cargamos el dataset que contiene los text_features
df = pd.read_excel('baseline.xlsx')

# Eliminamos el index de nuestro dataset
df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

# Revisamos las columnas
df.columns

Index(['tramp_id', 'sampling_date', 'lat', 'lon', 'municipality',
       'square_area', 'plantation_age', 'capture_count', 'state',
       'square_area_imputed', 'month', 'year', 'year_month', 'day_of_year_sin',
       'day_of_year_cos', 'day_of_week_sin', 'day_of_week_cos',
       'week_of_year_sin', 'week_of_year_cos', 'month_sin', 'month_cos',
       'critical_season', 'severity_encoded', 'distance_to_nearest_hotspot',
       'hotspots_within_5km', 'text_feature_location', 'text_feature_risk',
       'text_feature_capture', 'text_feature_plantation',
       'text_feature_all_things'],
      dtype='object')

In [21]:
# Utilizaremos solamente la informacion a partir de Julio de 2025 para esta entrega
# ya que de lo contrario tendríamos más de 1,000,000 de archivos para hacer el fine-tuning
year_mask = df['year'] > 2024
month_mask = df['month'] > 6
df = df[year_mask]
df = df[month_mask]

C:\Users\Delbert\AppData\Local\Temp\ipykernel_17416\1499313213.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[month_mask]


In [22]:
severity_dict = {
    0: 'sin riesgo',
    1: 'de riesgo leve',
    2: 'de riesgo moderado',
    3: 'de riesgo severo'
}

critical_season_dict = {
    0: 'normal',
    1: 'critica'
}


area_information_dict = {
    'original': " (area historica registrada correctamente)", 
    'radius_regressor' : "(el area en hectareas se calculo usando trampas cercanas)", 
    'median' : "(el area en hectareas se calculo usando la mediana de los datos de muestra)", 
    'same_trap_id_temporal' : " (valor del area en hectarea obtenido usando registros historicos)"
}

In [23]:
### Esta función permite crear los archivos jsonl que se usaran para hacer el
### fine-tuning de Phi-3.

def generate_jsonl(name, instruction_variants, input_col_name, out_col_name):

    records = []
    for _, row in df.iterrows():
        for instruction in instruction_variants:
            records.append({
                "instruction": instruction,
                "input": row[input_col_name],
                "output": row[out_col_name]
            })
            

    output_path = f"{name}"
    with open(output_path, "w", encoding="utf-8") as f:
        for record in records:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

    print(f"✅ Archivo JSONL generado correctamente: {output_path}")
    print(f"Total de ejemplos creados: {len(records)} (≈ {len(instruction_variants)} × {len(df)})")

#### Generamos la información de ubicación (Estado, Municipalidad, lat y lon, etc.)

In [24]:
# Empezamos a generar los archivos jsonl para usarlos en el fine-tuning de nuestro LLM 

def build_location_input(x):
    return (
        f"tramp_id: {x['tramp_id']}, "
        f"lat: {x['lat']}, lon: {x['lon']}, "
        f"sampling_date: {x['sampling_date'].strftime('%d-%m-%Y')}, "
        f"municipality: {x['municipality']}, "
        f"state: {x['state']}, "
        f"plantation_age: {x['plantation_age']}, "
        f"square_area_imputed: {x['square_area_imputed']}, "
    )
    

instruction_variants_for_location = [
    "Convierte los datos estructurados del reporte de trampa en una descripcion en lenguaje natural.",
    "Describe la ubicacion y las condiciones del cultivo de agave con base en los datos del reporte."
]

df["input_for_location"] =  df.apply(build_location_input, axis= 1)
df["output_for_location"] = df["text_feature_location"]

generate_jsonl('agave_location.jsonl', instruction_variants_for_location, 'input_for_location', 'output_for_location')

✅ Archivo JSONL generado correctamente: agave_location.jsonl
Total de ejemplos creados: 62030 (≈ 2 × 31015)


#### Generamos el set de instrucciones para tomar en cuenta el rieso presente en algunos predios

In [25]:
def build_risk_input(x):
    return (
        f"tramp_id: {x['tramp_id']}, "
        f"sampling_date: {x['sampling_date'].strftime('%d-%m-%Y')}, "
        f"severity_encoded: {x['severity_encoded']} ({severity_dict.get(x['severity_encoded'], 'Desconocido')}), "
        f"distance_to_nearest_hotspot: {x['distance_to_nearest_hotspot']} km, "
        f"hotspots_within_5km: {x['hotspots_within_5km']}"
    )

instruction_variants_for_risk = [
    "Analiza los datos de una trampa y genera una descripción del nivel de riesgo de infestación por picudo del agave.",
    "Describe el riesgo de infestación considerando la severidad y la distancia a focos severos."
]


df["input_for_risk"] =  df.apply(build_risk_input, axis= 1)
df["output_for_risk"] = df["text_feature_risk"]


generate_jsonl('agave_risk.jsonl', instruction_variants_for_risk, 'input_for_risk', 'output_for_risk')

✅ Archivo JSONL generado correctamente: agave_risk.jsonl
Total de ejemplos creados: 62030 (≈ 2 × 31015)


#### Tomamos en cuenta la información de captura por cada trampa que sobrevivió los filtros de año y fecha

In [26]:
def build_capture_input(x):
    return (
        f"tramp_id: {x['tramp_id']}, "
        f"sampling_date: {x['sampling_date'].strftime('%d-%m-%Y')}, "
        f"capture_count: {x['capture_count']}, "
        f"severity_encoded: {x['severity_encoded']} "
        f"({severity_dict.get(x['severity_encoded'], 'Desconocido')}), "
        f"critical_season: {x['critical_season']} "
        f"({critical_season_dict.get(x['critical_season'], 'No especificado')})"
    )

instruction_variants_for_capture = [
    "Describe en lenguaje natural los resultados de captura obtenidos por la trampa durante la temporada indicada.",
    "Genera una descripción completa de la cantidad de gorgojos capturados y el nivel de infestación detectado."
]

df["input_for_capture"] = df.apply(build_capture_input, axis=1)
df["output_for_capture"] = df["text_feature_capture"]

generate_jsonl('agave_capture.jsonl', instruction_variants_for_capture, 'input_for_capture', 'output_for_capture')

✅ Archivo JSONL generado correctamente: agave_capture.jsonl
Total de ejemplos creados: 62030 (≈ 2 × 31015)


#### Genramos el jsonl para describir a las plantaciones en función de los años de operación, área, etc.

In [27]:
def build_plantation_input(x):
    return (
        f"square_area_imputed: {x['square_area_imputed']} ha, "
        f"plantation_age: {x['plantation_age']} años, "
        f"capture_count: {x['capture_count']} gorgojos, "
        f"severity_encoded: {x['severity_encoded']} "
        f"({severity_dict.get(x['severity_encoded'], 'Desconocido')})"
    )
    
instruction_variants_for_plantation = [
    "Describe en lenguaje natural las características de la plantación de agave, incluyendo su tamaño, edad y nivel de infestación.",
    "Genera una descripción completa del estado de la plantación considerando el área, la edad y la cantidad de gorgojos capturados."
]

df["input_for_plantation"] = df.apply(build_plantation_input, axis=1)
df["output_for_plantation"] = df["text_feature_plantation"]

generate_jsonl('agave_plantation.jsonl', instruction_variants_for_plantation, 'input_for_plantation', 'output_for_plantation')

✅ Archivo JSONL generado correctamente: agave_plantation.jsonl
Total de ejemplos creados: 62030 (≈ 2 × 31015)


#### Por último, creamos un jsonl que contenga la combinación de todos los demás text features para dar más contexto a nuestro LLM.

In [28]:
def build_allThings_input(x):
    return (
        f"tramp_id: {x['tramp_id']}, "
        f"lat: {x['lat']}, lon: {x['lon']}, "
        f"sampling_date: {x['sampling_date'].strftime('%d-%m-%Y')}, "
        f"municipality: {x['municipality']}, state: {x['state']}, "
        f"square_area_imputed: {x['square_area_imputed']} ha, "
        f"plantation_age: {x['plantation_age']} años, "
        f"capture_count: {x['capture_count']} gorgojos, "
        f"severity_encoded: {x['severity_encoded']} "
        f"({severity_dict.get(x['severity_encoded'], 'Desconocido')}), "
        f"distance_to_nearest_hotspot: {x['distance_to_nearest_hotspot']} km, "
        f"hotspots_within_5km: {x['hotspots_within_5km']}"
    )
    
instruction_variants_for_all_things = [
    "Genera una descripción completa de la plantación, la ubicación de la trampa y el nivel de riesgo de infestación combinando toda la información disponible.",
    "Describe en lenguaje natural los detalles de la trampa, la plantación y las condiciones de infestación, incluyendo ubicación, distancia a focos y severidad.",
    "Redacta un resumen integral que combine los datos de captura, características de la plantación y proximidad a focos severos de infestación."
]


df["input_for_allThings"] = df.apply(build_allThings_input, axis=1)
df["output_for_allThings"] = df["text_feature_all_things"]

generate_jsonl('agave_allThings.jsonl', instruction_variants_for_all_things, 'input_for_allThings', 'output_for_allThings')

✅ Archivo JSONL generado correctamente: agave_allThings.jsonl
Total de ejemplos creados: 93045 (≈ 3 × 31015)


#### Función para unir todos los text_features en un solo jsonl

Esta función es opcional, pero recomendamos generar los archivos por separado ya que durante el proceso de fine-tuning fue necesario contextualizar al LLM por `chunks`o pedazos.

Esto permitió que el tiempo de fine-tuning fuera menor en comparación de cargar un archivo con todos los features, y habilitó también la mezcla de distintas `instructions` y `outputs` sin la necesidad de usar shuffling o selecciones aleatorias de data.

In [ ]:
# import json

# files_to_merge = [
#     "/content/drive/MyDrive/LLM-training/agave_location.jsonl",
#     "/content/drive/MyDrive/LLM-training/agave_risk.jsonl",
#     "/content/drive/MyDrive/LLM-training/agave_capture.jsonl",
#     "/content/drive/MyDrive/LLM-training/agave_capture.jsonl",
#     "/content/drive/MyDrive/LLM-training/agave_allThings.jsonl"
# ]

# output_path = "/content/drive/MyDrive/LLM-training/agave_combined_multitask.jsonl"

# with open(output_path, "w", encoding="utf-8") as outfile:
#     for path in files_to_merge:
#         with open(path, "r", encoding="utf-8") as infile:
#             for line in infile:
#                 outfile.write(line)

# print(f"✅ Combined file saved to: {output_path}")

### Ahora sí probaremos nuestro LoRA y Phi-3 personalizado

Ahora cargamos los mismos parámetros que usamos para llevar a cabo el fine-tune de `Phi-3`.

En esta sección es importante mencionar que para reducir el tiempo de fine-tuning en Google-Colab, decidimos cuantizar a `Phi-3` en 4bits para generar los LoRA adapters en un tiempo prudente. Con esto, aunque estos LoRA adapters fueron creados con base en la cuantización de 4bits de Phi-3, es posible cargar a Phi-3 cuantizado en 8bits y acoplarle los LoRa que generamos.

Por último, en este punto recomendamos reiniciar el kernel de Python y cargar de nuevo las librerías a utilizar. De esta manera podemos asegurar que la memoria RAM de nuestra GPU estará al mínimo.

In [30]:
model.cpu()
del model

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [3]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [4]:


# Modelo local 
model_path = "D:/LLM Models/microsoft-phi-3-mini-4k-instruct"

# LoRA adapters generados con el archivo LL_Fine_Tune_agave.ipynb
lora_path = "D:/LLM Models/agave_V001/agave_baseline_phi3_V01"   

# 8 bits de cuantizacion
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

# Usamos el tokenizer de Phi-3
tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
)



`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.11s/it]


In [5]:
# Implementamos los pesos LoRA a Phi-3
model = PeftModel.from_pretrained(base_model, lora_path)

# Colocamos el  modelo en evaluacion para que no cambie los pesos por cada prompt que
# se le envíe
model.eval()

# Definimos el pipeline de nuevo
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    return_full_text=False,
)

Device set to use cuda:0


Ahora, durante el fine-tuning de Phi-3 se usó el siguiente template de conversación:

```
<|user|>
`Pregunta o  prompt del usuario`.<|end|>
<|assistant|>
`Respusta de Phi-3 al usar el Chat template`
```

Esto se hizo con base en la documentación mostrada por (Voigt, 2025), y por lo tanto para que nuestro modelo fine-tuned funcione correctamente, debemos convertir nuestros prompts a esta forma.

In [6]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [{"role": "user", "content": sentence}]
    prompt = tokenizer.apply_chat_template(
        converted_sample, tokenize=False, add_generation_prompt=True
    )
    return prompt

In [7]:


def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
  
  tokenized_input = tokenizer(
  prompt, add_special_tokens=False, return_tensors="pt").to(model.device)
  model.eval()
  gen_output = model.generate(**tokenized_input,
  eos_token_id=tokenizer.eos_token_id,
  max_new_tokens=max_new_tokens)
  output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)

  return output[0]

### Con `prompt_1`

Notamos que ahora con `prompt_1`, nuestro LLM agrega contexto tomando en cuenta a México.

In [8]:
sentence = prompt_1
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

print(generate(model, tokenizer, prompt))

<|user|>
Describe al gorgojo del agave brevemente<|end|>
<|assistant|>

<|user|> Describe al gorgojo del agave brevemente<|end|><|assistant|> El gorgojo del agave es una especie de insecto escamudo perteneciente a la familia de los Chrysomelidae, comúnmente conocida como la lagartija del agave. Se distribuye principalmente en México y América Central. Este insecto es conocido por su capacidad


### Con `prompt_2`

In [9]:
sentence = prompt_2
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

print(generate(model, tokenizer, prompt))

<|user|>
Describe la planta de agave azul brevemente<|end|>
<|assistant|>

<|user|> Describe la planta de agave azul brevemente<|end|><|assistant|> La planta de agave azul, conocida científicamente como Agave tequilana, es una especie de planta suculenta nativa de México. Es ampliamente cultivada por su capacidad de producir agave, un ingrediente clave en la elabora Convierte


Con `prompt_3`

In [10]:
sentence = prompt_3
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

print(generate(model, tokenizer, prompt))

<|user|>
Describe como es un predio de agave azul y cuales riesgos presenta brevemente<|end|>
<|assistant|>

<|user|> Describe como es un predio de agave azul y cuales riesgos presenta brevemente<|end|><|assistant|> Un predio de agave azul, conocido como maguey, se caracteriza por su vegetación xerófila y su estructura de tramp_id: 11390-11909, ubicado en la región de Jalisco, México. La


Al mencionar un ID de trampa que se usó para entrenar el modelo, es posible observar que ahora nuestro ChatBot puede conocer la información general de la misma.

En esta etapa, aunque el ChatBot no contestó con la información de la trampa en sí, fue posible obtener algo de información de él. Ahora bien, es indispensable llevar a cabo el fine-tune de nuestro LLM con más muestras de nuestro dataset, tomando en cuenta que se requiere de más tiempo para que se procese correctamente la creación de LoRA adapters.

In [11]:
sentence = "gorgojos capturados por tramp id: 1387"
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

print(generate(model, tokenizer, prompt))

<|user|>
gorgojos capturados por tramp id: 1387<|end|>
<|assistant|>

<|user|> gorgojos capturados por tramp id: 1387<|end|><|assistant|> tramp_id: 1387, lat: 20.4713, lon: -104.173, sampling_date: 15-07-2025, municipality: JUCHITLAN, state: JALISCO


Al trabajar con un prompt más específico, notamos que ahora el LLM puede recuperar correctamente las variables `lat`, `lon`, `sampling_date` y `state` (este ultimo fue truncado por la cantidad maxima de Tokens a generar).

Con esto confirmamos que nuestro modelo "aprendió" la forma de "escribir" o predecir las respuestas. Es importante notar que el prompt utilizado fue bastante parecido a lo que se usó como `instruction` en su entrenamiento.

In [12]:
sentence = "Genera una descripción completa de la plantación, la ubicación de la trampa y el nivel de riesgo de infestación combinando toda la información disponible para tramp_id: 1387."
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

print(generate(model, tokenizer, prompt))

<|user|>
Genera una descripción completa de la plantación, la ubicación de la trampa y el nivel de riesgo de infestación combinando toda la información disponible para tramp_id: 1387.<|end|>
<|assistant|>

<|user|> Genera una descripción completa de la plantación, la ubicación de la trampa y el nivel de riesgo de infestación combinando toda la información disponible para tramp_id: 1387.<|end|><|assistant|> tramp_id: 1387

lat: 20.483333

lon: -103.083333

sampling_date: 15-07-2025

state: JALA


Al especificar mucho más nuestra consulta con base en la estructura `instruction`-`input`-`output` generados, es posible obtener respuestas más concretas.

En este caso, obtener la respuesta correcta: `JALISCO`.

In [13]:
sentence = "Eres un asistente agricultor que permite consultar información sobre la ubicación de trampas. Las trampas se identifican con un tramp_id. Dime en qué estado se encontraba la tramp_id: 1402."
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

print(generate(model, tokenizer, prompt))

<|user|>
Eres un asistente agricultor que permite consultar información sobre la ubicación de trampas. Las trampas se identifican con un tramp_id. Dime en qué estado se encontraba la tramp_id: 1402.<|end|>
<|assistant|>

<|user|> Eres un asistente agricultor que permite consultar información sobre la ubicación de trampas. Las trampas se identifican con un tramp_id. Dime en qué estado se encontraba la tramp_id: 1402.<|end|><|assistant|> La tramp_id: 1402 se encuentra en el estado de Jalisco.<|end|><|endoftext|>


En este punto, notamos que es necesario un sistema RAG para evitar que nuestro LLM genere respuestas que no tengan mucha relación con lo que se le preguntó, y sobre todo que el formato de las mismas sea coherente.

In [14]:
sentence = "Genera una descripción completa de la cantidad de gorgojos capturados y el nivel de infestación detectado de la tramp_id: 1387 y cuántos gorgojos capturó el 15-07-2024."
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

print(generate(model, tokenizer, prompt))

<|user|>
Genera una descripción completa de la cantidad de gorgojos capturados y el nivel de infestación detectado de la tramp_id: 1387 y cuántos gorgojos capturó el 15-07-2024.<|end|>
<|assistant|>

<|user|> Genera una descripción completa de la cantidad de gorgojos capturados y el nivel de infestación detectado de la tramp_id: 1387 y cuántos gorgojos capturó el 15-07-2024.<|end|><|assistant|> Tramp_id: 1387

Lat, Long: 20.900833, -104.183333

Precipitation: 0.00 mm

Temperature: 26.00 °


# Sistema RAG

Ahora que ya hicimos un fine-tuning sobre `Phi-3`, aunque este fue bastante superficial porque solo usamos 12,500 registros `instruction`-`input`-`output`, podemos indicar que nuestro LLM ahora "escribirá" de manera más propia la información con base en las respuestras de los prompts anteriores.

Por lo anterior, lo que haremos ahora será imlementar un sistema RAG el cual le dará las siguientes características a nuetro  LLM:

- Lo volverá más estable al responder. Por medio del sistema RAG, contextos y demás, será posible reforzar la forma en que el modelo responderá y demostrará la información.
- Permite darle instrucciones al LLM para no halucinar. Por ejemplo, es posible indicar que si la información no se encuentra disponible en el contexto, que no responda la pregunta.

Con esto, montar el sistema RAG sobre nuestro Phi-3 con fine-tuning permite que, aparte de que nuestro Bot hable más propiamente, pueda usar información histórica para responder dudas comunes.

In [15]:
# Cargamos el Manual Operativo como un documento para consulta del bot.
loader = PyPDFLoader(r'C:/Users/Delbert/Documents/Maestria/Proyecto Integrador/Data/Manual_Operativo_Agave.pdf')
docs = loader.load()

print(f"Se cargaron {len(docs)} paginas desde el Manual Operativo.")
print(f"Y los primeros 100 caracteres son los siguientes:\n")
print(docs[0].page_content[:100])

Se cargaron 44 paginas desde el Manual Operativo.
Y los primeros 100 caracteres son los siguientes:

DIRECCIÓN DE PROTECCIÓN FITOSANITARIA 
  Clave: MOP-DPF-PRAV 
  Versión: 4 
 
I 
  Emisión: 04/2017 


In [16]:
# Cargamos todos los text_features previamente generados que podrán ser consultados de nuevo.

from pathlib import Path
import json

def load_jsonl_as_documents(jsonl_paths):

    all_docs = []

    for file_path in jsonl_paths:
        path = Path(file_path)
        if not path.exists():
            print(f"No se encontró el archivo siguiente: {file_path}")
            continue

        with open(path, "r", encoding="utf-8") as f:
            for line_num, line in enumerate(f, start=1):
                try:
                    record = json.loads(line.strip())
                    # Usamos de nuevo la estructura de Alpaca
                    text = (
                        f"Instrucción: {record.get('instruction', '')}\n"
                        f"Entrada: {record.get('input', '')}\n"
                        f"Respuesta: {record.get('output', '')}"
                    )

                    # Tomamos metadata
                    doc = Document(
                        page_content=text,
                        metadata={
                            "source": path.name,
                            "line": line_num
                        }
                    )
                    all_docs.append(doc)
                except json.JSONDecodeError as e:
                    print(f"Se dio un error en la linea {line_num} en {path.name}: {e}")
                    continue

    print(f"Se cargaron {len(all_docs)} documentos de {len(jsonl_paths)} archivos JSONL.")
    return all_docs

jsonl_files = [
    "agave_allThings.jsonl",
    "agave_capture.jsonl",
    "agave_location.jsonl",
    "agave_plantation.jsonl",
    "agave_risk.jsonl"
]

docs_jsonl = load_jsonl_as_documents(jsonl_files)
print(docs_jsonl[0].page_content[:300])

Se cargaron 341165 documentos de 5 archivos JSONL.
Instrucción: Genera una descripción completa de la plantación, la ubicación de la trampa y el nivel de riesgo de infestación combinando toda la información disponible.
Entrada: tramp_id: 6538-145, lat: 20.92757, lon: -103.8551, sampling_date: 04-07-2025, municipality: TEQUILA, state: JALISCO, square


#### Vector Store

Una parte ***importante*** de un sistema RAG son los embeddings que permitirán que nuestro LLM relacione de mejor manera las palabras. Estos vectores no son más que "pesos" que relacionan palabras entre sí y permitirán que el LLM pueda predecir de mejor manera las respuestas.

Estos embeddings terminan siendo como el "alma" que permite que nuestro Bot no alucine y tampoco conteste algo que no sea real. En este paso, debido la cantidad de documentos, es normal esperar entre 60 a 120 minutos de generación de embeddings. Este proceso se puede acelerar por medio de Google Colab.

In [17]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunked_docs = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    encode_kwargs={"normalize_embeddings": True}
)

C:\Users\Delbert\AppData\Local\Temp\ipykernel_7524\854352266.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


El siguiente código se debe des-comentar para generar los embeddings de forma local. Solo debemos tener en mente que es un proceso que conlleva de 60 a 120 minutos para generarlos.

En nuestro caso, los generamos una sola vez y a partir de eso solo los cargamos.

In [ ]:
# vectorstore = FAISS.from_documents(chunked_docs, embedding=embeddings)
# vectorstore.add_documents(docs_jsonl)

# # Guardamos la vector store para cargas mas rapidas despues
# vectorstore.save_local("rag_faiss_store")

In [18]:

# Para agilizar otras  corridas, podemos volver a cargar los embeddings generados a partir de la última sesión
vectorstore = FAISS.load_local("rag_faiss_store", embeddings, allow_dangerous_deserialization=True)

In [19]:
vs = FAISS.load_local(
    "C:/Users/Delbert/Documents/Maestria/Proyecto Integrador/Avance 1/Tecnologico-Monterrey-Proyecto-Integrador-equipo-36/Baseline/rag_faiss_store",
    embeddings,
    allow_dangerous_deserialization=True
)

print("Total docs:", len(vs.docstore._dict))

Total docs: 619918


Ahora bien, como nuestro LLM tuvo un fine-tuning con base en 2 templates (`instruction, input & output` y el `chat-template` de Phi-3), es importante que nuestro sistema RAG pueda encapsular las consultas que se le hagan de forma que el LLM lo entienda.

En el caso en que cambiemos el LLM utilizado, deberíamos de generar una nueva clase para lograr lo mismo. Por lo tanto, en este punto, ya estamos practicamente comprometidos con continuar con Phi-3 ya que:

1. Hicimos un proceso de fine-tuning sobre `Phi-3` por varias horas
2. Creamos una clase para hacer wraping del sistema RAG para que `Phi-3` comprenda los queries haciendo uso  del template `ChatBot`.
3. Realizar de nuevo estos entrenamientos sobre otro modelo puede ser computacionalmente más caro. Por ejemplo, `Mistral-7B` aun cuantizado en 4bits consume bastante más VRAM que `Phi-3` cuantizado en 8bits


Por último, configuramos nuestro LLM para producir un máximo de 32 tokens de salida. Si se quiere tener respustas más largas, debemos definir un `max_new_tokens` distinto.

In [20]:
class Phi3ChatTemplateLLM(LLM):
    model: Any = Field(exclude=True)
    tokenizer: Any = Field(exclude=True)
    max_new_tokens: int = 32
    skip_special_tokens: bool = True

    @property
    def _llm_type(self) -> str:
        return "phi3-chat-template"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs: Any) -> str:
        chat_prompt = gen_prompt(self.tokenizer, prompt)
        out = generate(
            self.model,
            self.tokenizer,
            chat_prompt,
            max_new_tokens=self.max_new_tokens,
            skip_special_tokens=self.skip_special_tokens
        )
        
        return out


Cargamos el modelo custom (contenido en la clase) para hacer pruebas.

Usaremos el tokenizer incluido con la versión original de `Phi-3` para ahorrarnos el proceso de definición de tokens (por palabra, oración, párrafo, etc.).

In [21]:
custom_llm = Phi3ChatTemplateLLM(model=model, tokenizer=tokenizer)

Configuramos el sistema `retriever` para implementar el RAG.

Y generamos un prompt el cual, por medio de distintas pruebas, fue posible definir para obtener respuestas concretas de nuestro bot.

In [22]:
my_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Eres un asistente operativo especializado en el picudo del agave. "
        "Usa **exclusivamente** la información del CONTEXTO para responder. "
        "Si la respuesta no aparece en el contexto, di exactamente: "
        "\"No aparece en el contexto.\" No inventes datos.\n\n"
        "CONTEXTO:\n{context}\n\n"
        "PREGUNTA:\n{question}\n\n"
        "RESPUESTA DE ASISTENTE:"
    ),
)



retriever = vs.as_retriever(search_kwargs={"k": 8})

rag_chain = RetrievalQA.from_chain_type(
    llm=custom_llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": my_prompt},
    return_source_documents=False
)


Y ahora generamos un par de prompts.

In [23]:
query = "¿Qué es un agave?"
response = rag_chain.invoke({"query": query})

raw_output = response["result"]


# Limpiamos la respuesta
if "RESPUESTA DE ASISTENTE:" in raw_output:
    clean_output = raw_output.split("RESPUESTA DE ASISTENTE:")[-1].strip()
else:
    clean_output = raw_output.strip()

print(clean_output)


Un agave es una planta suculenta originaria de México y América Central, perteneciente a la familia de las agaváceas. Se


Por último, dejamos las versiones de Torch y CUDA que utilizamos

In [24]:
# Version de Pytorch
print("PyTorch version:", torch.__version__)

# 
print("CUDA available:", torch.cuda.is_available())

PyTorch version: 2.5.1+cu121
CUDA available: True


# Conexión a WhatsApp

Por medio de Twilio y su integración como ISV de Meta, es posible hacer pruebas en una sandbox la cual no tiene costo, pero sí un límite diario de mensajes.

Con lo anterior, debemos instalar varias librerías en nuestro ambiente:

- Twilio
- Flask (esto servirá para recibir los mensajes desde Twilio por medio de un webhook)
- dotenv (para cargar las variables de entorno para autenticarse en Twilio)
- Ngrok (aunque no es un módulo en este notebook, se tuvo que instalar para hacer hosting de nuestra aplicación de Flask en un endpoint que Ngrok genera)

In [25]:
from twilio.twiml.messaging_response import MessagingResponse
from flask import Flask, request, Response
from twilio.rest import Client
from dotenv import load_dotenv
import requests


load_dotenv(dotenv_path="chatbot.env")

account_sid = os.getenv('account_sid_2')  # Variable de entorno para usar la API de Twilio
auth_token  = os.getenv('auth_token_2')   # Variable de entorno para autenticarse ante la API de Twilio
FROM_NUMBER  = ""                         # Numero del sandbox que Twilio proporciona
client = Client(account_sid, auth_token ) # Intanciamos el objeto de Twilio para probar nuestro ChatBot 

## Flujo principal para reporte de predios infestados con el gorgojo del agave

Definimos un diccionario para conocer el estado de cada chat en el prototipo.

In [26]:
# Global session storage (temporary — resets if the app restarts)
user_session = {}

#### Envío de mensajes por medio de Twilio-WhatsApp-Sandbox

Definimos una función para enviar mensajes sin formato a WhatsApp, los cuale se conocen como FreeForm.

In [28]:
def send_whatsapp_message(From, To, message):

    try:
        
        msg = client.messages.create(
            from_=From,
            to=To,
            body=message
        )
        # print(f"Message sent: {msg.sid}") # Descomentar esta linea para ver el ID del mensaje enviado por WhatsApp
        return msg.sid
    
    except Exception as e:
        
        print(f"Error sending message: {e}")
        return None

Definimos también una función para mostrar el menú principal.

In [29]:
def send_menu(From, To):
    
    menu_message = (
        "👋 *Bienvenido, será un gusto atenderte.*\n\n"
        "¿Qué te interesa llevar a cabo?\n\n"
        "🅰️ *Hacer un reporte de un predio infectado*\n"
        "🅱️ *Preguntar por información*"
        "\n\n"
        "Actualmente, solo estas 2 opciones tenemos disponibles."
    )

    try:
        msg = client.messages.create(
            from_=From,
            to=To,
            body=menu_message
        )
        # print(f"✅ Se envio correctamente el menu: {msg.sid}") # Descomentar esta linea para ver el ID del mensaje enviado por WhatsApp
        return msg.sid
    except Exception as e:
        print(f"❌ Error enviando menu: {e}")
        return None

Se define también el flujo principal para recabar información.

Esto no incluye algo de NLP ya que el flujo se encuentra bastante definido:

- Debemos solicitar la ubicación
- Una fotografía de la infestacion o predio
- La severidad que el usuario considera

In [30]:
def report_message_flow( sender, state, req ):

	# Cargamos el mensaje
	message = request.form.get("Body", "").strip().lower() # Texto enviado por el usuario
	label   = request.form.get("Label")  				   # Label de la ubicacion enviada
	lat = request.form.get("Latitude") 					   # Si envian ubicacion, cargamos la latitud
	lon = request.form.get('Longitude') 				   # Si envian ubicacion, cargamos la longitud
	address = request.form.get('Address') 				   # Tomamos la dirección generada por WhatsApp
	photo_url = req.form.get('MediaUrl0')                  # Si envia una fotografia, obtendremos un URL de la misma
	num_media = int(req.form.get('NumMedia', 0))           # Permite saber si enviaron un mensaje con contenido multimedia
 
	response_text = ""
	next_state = state
 
	if state == "saludo":
		if (message == "a") or (message == "🅰️"):
			response_text = ("Gracias por tu proactividad. Te guiaré para reportar el predio. Primero, envía la **ubicación** del lote afectado.")
			next_state  = "ask-location"

		elif (message == "b") or (message == "🅱️"):
			response_text = ("Claro, dime, ¿cuál es tu duda?")
			next_state = "NLP-Chatbot"
   
	elif state == "ask-location":
		if lat and lon:
			user_session[sender]["data"]["lat"]  = lat
			user_session[sender]["data"]["lon"] = lon
			user_session[sender]["data"]["address"]   = address
			response_text = (
				"📍 Ubicación recibida *correctamente*.\n"
				"Ahora, por favor envía una **foto** del lote o planta afectada."
			)
			next_state = "ask_photo"
		else:
			response_text = ("Parece que la información que enviaste no es correcta. Por favor, usa la función de ubicación de WhatsApp.")

	elif state == "ask_photo":
		if num_media > 0:
			photo_url = req.form.get('MediaUrl0')
			user_session[sender]["data"]["photo_url"] = photo_url
			response_text = (
				"📸 Foto recibida correctamente.\n"
				"Por último, clasifica el **nivel de riesgo** que observas: Alto, Medio o Bajo."
			)
			next_state = "ask_risk"
		else:
			response_text = (
				"⚠️ No se detectó una foto. Por favor envía una imagen del lote afectado."
			)

	elif state == "ask_risk":
    
		if message in ["alta", "media", "baja", "alto", "medio", "bajo"]:
			user_session[sender]["data"]["user_risk_assesment"] = message.capitalize()
			data = user_session[sender]["data"]
			response_text = (
				f"✅ Gracias por tu reporte.\n"
				f"📍 Ubicación: ({data['lat']}, {data['lon']})\n"
				f"📸 Foto: Confirmada\n"
				f"🚨 Riesgo: {data['user_risk_assesment']}\n\n"
				"¿Esta información es correcta? Responde con *Sí* o *No*."
			)
			next_state = "confirmation"

		else:
			response_text = (
				"Por favor indica el nivel de riesgo como: Alta, Media o Baja."
			)

	elif state == "confirmation":
		msg = message.strip().lower()

		if re.search(r'^\s*s[ií]\s*$', msg):
			response_text = (
				"🌾 Tu reporte ya fue registrado. ¡Gracias por tu colaboración!"
			)
			print(user_session[sender])
		elif re.search(r'^\s*no\s*$', msg):
			response_text = "❌ Entendido. Tu reporte no se registrará."
		else:
			response_text = "Por el momento, solo podemos aceptar respuestas como 'Sí' o 'No'."
	else:
		response_text = ("Por el momento, solo podemos aceptar respuestas como 'Sí', 'No', etc. ")

	return response_text, next_state

Creamos la función principal para recibir mensajes desde la sandbox de Twilio. En este punto tenemos 2 formas de atender al usuario:

- Seguir el flujo de un reporte de infestación.
- Permitir al usuario hacer preguntas sobre el manual de operaciones y data histórica de focos de infestación.

In [ ]:
# reiniciamos user_session para hacer pruebas

user_session = {}

app = Flask(__name__)

@app.route("/reply_whatsapp", methods=['POST'])

def reply_whatsapp():

	sender  = request.form.get("From") # Guardamos quien nos escribio
	message = request.form.get("Body", "").strip().lower() # Tomamos el texto del mensaje para mostrarlo en la consola
	resp = MessagingResponse()
 
	print(f"Mensaje recibido: --> {message}")

	# si no tenemos un chat registrado con el usuario, procedemos a agregarlo al diccionario de sesiones
	if sender not in user_session:
		user_session[sender] = {"state": "saludo",
								"data" : {
									"lat" : None, 
        							"lon" : None,
									"address" : None,
									"photo_url" : None,
									"user_risk_assesment": None,
									"price" : None
								}
              					}

		send_menu(FROM_NUMBER, sender)
		resp.message("Quedo a la espera de tu respuesta.")
		return Response(str(resp), mimetype="text/xml")
	
	# Si el usuario decidió hacer uso del feature de preguntas, usamos el LLM para contestar con el sistema RAG
	if user_session[sender]['state'] == "NLP-Chatbot":
		print("Generando respuesta con el LLM")
		query = message
		response = rag_chain.invoke({"query": query})

		raw_output = response["result"]

		if "RESPUESTA DE ASISTENTE:" in raw_output:
			clean_output = raw_output.split("RESPUESTA DE ASISTENTE:")[-1].strip()
		else:
			clean_output = raw_output.strip()
		
		response_text = clean_output
		print(f"{response_text}")

	# Si el usuario decidió seguir con el flujo de reporte, continuamos con el mismo
	else:
		
		response_text, next_state = report_message_flow(sender, user_session[sender]['state'], request)
		user_session[sender]["state"] = next_state
		
	
	# Enviamos la respuesta segun lo que el usuario decidio al inicio en el menu
	resp.message(response_text)
	
	return Response(str(resp), mimetype='text/xml')


if __name__ == "__main__":
	app.run(port=3000)